# Genetic algorithm model calibration

This notebook applies a genetic algorithm to calibrate wall decay coefficients using data from each water quality sensing period.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.colors
default_colors = plotly.colors.qualitative.Plotly
from bayesian_wq_calibration.epanet import build_model, epanet_simulator, sensor_model_id
from bayesian_wq_calibration.ga import evaluate, decision_variables_to_dict
from bayesian_wq_calibration.constants import TIMESERIES_DIR, RESULTS_DIR
from bayesian_wq_calibration.plotting import plot_network
from bayesian_wq_calibration.data import bulk_temp_adjust
import random
from deap import base, creator, tools, algorithms
from functools import partial
import multiprocessing
import time

### Load data and build water model

Load operational data for selected sensing period.

In [29]:
data_period = 10 # 20 calibration events (as at 30 October 2024)
try:
    flow_df = pd.read_csv(TIMESERIES_DIR / f"processed/{str(data_period).zfill(2)}-flow.csv")
    pressure_df = pd.read_csv(TIMESERIES_DIR / f"processed/{str(data_period).zfill(2)}-pressure.csv")
    wq_df = pd.read_csv(TIMESERIES_DIR / f"processed/{str(data_period).zfill(2)}-wq.csv", low_memory=False)
    cl_df = wq_df[wq_df['data_type'] == 'chlorine']
    print(f"Calibrating wall model decay coefficients for data period: {data_period}...")
except:
    print(f"Data period {data_period} does not exist.")

Calibrating wall model decay coefficients for data period: 10...


Split sensing data into train and test datasets.

In [30]:
n_total = len(flow_df['datetime'].unique())
n_train = 2 * 24 * 4

train_range = range(n_train)
train_datetime = flow_df['datetime'].unique()[list(train_range)]

test_range = range(n_train, n_total)
test_datetime = flow_df['datetime'].unique()[list(test_range)]

total_range = range(n_total)
total_datetime = flow_df['datetime'].unique()[list(total_range)]

Set grouping type and initialize decay coefficients.

In [44]:
grouping = 'roughness'
bulk_coeff = -0.5 # day^-1 (from BW)
field_temp = wq_df[wq_df['data_type'] == 'temperature']['mean'].mean()
bulk_coeff = bulk_temp_adjust(bulk_coeff, field_temp)

if grouping == 'single':
    n_coeffs = 1
    bounds = [(-5.0, 0.0)] # single wall decay coefficient
elif grouping == 'material':
    n_coeffs = 3
    bounds = [(-5.0, 0.0), (-2.0, 0.0), (-1.0, 0.0)] # variable order: metallic, cement, plastic_unknown
elif grouping == 'material-diameter':
    n_coeffs = 4
    bounds = [(-5.0, 0.0), (-3.0, 0.0), (-2.0, 0.0), (-1.0, 0.0)] # variable order: metallic_less_than_150, metallic_greater_than_150, cement, plastic_unknown
elif grouping == 'roughness':
    n_coeffs = 6
    bounds = [(-5.0, 0.0), (-5.0, 0.0), (-3.0, 0.0), (-3.0, 0.0), (-2.0, 0.0), (-2.0, 0.0)] # variable order: less_than_50, between_50_and_65, between_65_and_80, between_80_and_100, between_100_and_120, greater_than_120
plot_network(grouping=grouping)

In [45]:
def enforce_bounds(individual, bounds):
    for i, (lb, ub) in enumerate(bounds):
        if individual[i] < lb:
            individual[i] = lb
        elif individual[i] > ub:
            individual[i] = ub
    return individual

Build water model via WNTR module.

In [46]:
demand_resolution = 'wwmd'
wn_train = build_model(flow_df[flow_df['datetime'].isin(train_datetime)], pressure_df[pressure_df['datetime'].isin(train_datetime)], cl_df[cl_df['datetime'].isin(train_datetime)], sim_type='chlorine', demand_resolution=demand_resolution, bulk_coeff=bulk_coeff)

### Initialize GA using DEAP module.

Assign default parameters.

In [47]:
popsize = 50
ngen = 50
cxpb = 0.7
mutpb = 0.1
tournsize = 3
hofsize = 1

Set up DEAP structures.

In [48]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # minimize objective function
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()
def generate_individual():
    return [random.uniform(lb, ub) for lb, ub in bounds]
toolbox.register("individual", tools.initIterate, creator.Individual, generate_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.2)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("select", tools.selTournament, tournsize=tournsize)
# pool = multiprocessing.Pool()
# toolbox.register("map", pool.map)

low = [b[0] for b in bounds]
up = [b[1] for b in bounds]
toolbox.register("mutate", tools.mutPolynomialBounded, low=low, up=up, eta=20, indpb=0.2)

### Run GA

In [49]:
def main(wn, cl_df, grouping):
    
    start_time = time.time()
    
    # preload data into the evaluation function
    evaluation_function = partial(evaluate, wn=wn, cl_df=cl_df, grouping=grouping)
    toolbox.register("evaluate", evaluation_function)

    # define statistics
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("max", np.max)
    stats.register("mean", np.mean)
    stats.register("min", np.min)

    # initialize GA parameters
    pop = toolbox.population(n=popsize)
    hof = tools.HallOfFame(hofsize)
    logbook = tools.Logbook()

    # set early stopping criteria
    stagnant_gens = 0
    max_stagnant_gens = 5
    threshold = 1e-5 

    # run GA
    for g in range(ngen):
        
        if g != 0:
            pop = toolbox.select(pop, k=len(pop))
            pop = algorithms.varAnd(pop, toolbox, cxpb=cxpb, mutpb=mutpb)
            
            for ind in pop:
                enforce_bounds(ind, bounds)
        
        # evaluate the individuals with invalid fitness
        invalids = [ind for ind in pop if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalids)
        for ind, fit in zip(invalids, fitnesses):
            ind.fitness.values = fit

        # update hall of fame and compile statistics
        hof.update(pop)
        record = stats.compile(pop)
        logbook.record(gen=g, nevals=len(invalids), **record)
        if g == 0:
            print(logbook.stream)
        else:
            print(logbook.stream.splitlines()[-1])

        # retrieve current best fitness from hall of fame
        current_best_fitness = hof[0].fitness.values[0]
        
        # initialize previous_best_fitness at the end of the first generation's evaluation
        if g == 0:
            previous_best_fitness = current_best_fitness

        # calculate percent difference for early stopping
        percent_difference = abs(current_best_fitness - previous_best_fitness) / max(abs(previous_best_fitness), 1e-10)
        
        # print(f"Gen {g}: Current best fitness = {current_best_fitness}, Previous best fitness = {previous_best_fitness}, Percent difference = {percent_difference}")

        # check if improvement exceeds threshold
        if percent_difference > threshold:
            previous_best_fitness = current_best_fitness
            stagnant_gens = 0
        else:
            stagnant_gens += 1
    
        # early stopping if no improvement
        if stagnant_gens >= max_stagnant_gens:
            print(f"Terminating GA @ generation {g} due to solution quality stagnation. Best fitness: {current_best_fitness}")
            break
        
    cpu_time = time.time() - start_time 
    
    return pop, logbook, hof, cpu_time


In [ ]:
pop, log, hof, cpu_time = main(wn_train, cl_df[cl_df['datetime'].isin(train_datetime)], grouping)

gen	max      	mean     	min      	nevals
0  	0.0519685	0.0418162	0.0272572	50    
1  	0.0508919	0.0376494	0.0268358	34    
2  	0.0469904	0.0336092	0.0118579	37    
3  	0.0440567	0.0286869	0.00907487	39    
4  	0.0437166	0.0250208	0.0069489 	36    
5  	0.0311038	0.017216 	0.00378386	35    
6  	0.0238541	0.0125685	0.0025237 	41    
7  	0.0202266	0.0088906	0.0025237 	40    
8  	0.0159948	0.00602533	0.00214116	32    
9  	0.0156748	0.00419517	0.00189911	33    
10 	0.00467624	0.00271823	0.00188522	41    
11 	0.00353998	0.00222128	0.00186234	41    
12 	0.0022843 	0.00202158	0.0018258 	36    
13 	0.00302629	0.00198372	0.00178294	33    
14 	0.00416082	0.00195129	0.00171173	41    
15 	0.00285774	0.00189829	0.00174334	36    
16 	0.00241394	0.0018518 	0.00174334	40    
17 	0.00234815	0.00181499	0.00172378	40    
18 	0.00186795	0.00176256	0.00169316	41    
19 	0.00441936	0.00179543	0.00168271	44    
20 	0.00197858	0.00172607	0.00165829	41    
21 	0.00441908	0.00175614	0.00163641	32    
22 	0.003373

Get train and test MSE results.

In [ ]:
# get optimized decay coefficients
wall_coeffs_opt = decision_variables_to_dict(grouping, hof[0])
print(wall_coeffs_opt)

# model simulation
wn = build_model(flow_df, pressure_df, cl_df, sim_type='chlorine', demand_resolution='wwmd', grouping=grouping, wall_coeffs=wall_coeffs_opt)
sim_results = epanet_simulator(wn, 'chlorine', cl_df)
cl_sim = sim_results.chlorine

# obtain simulated data @ sensor nodes
sensor_data = sensor_model_id('wq')
cl_sim = cl_sim[sensor_data['model_id'].unique()]
name_mapping = sensor_data.set_index('model_id')['bwfl_id'].to_dict()
cl_sim = cl_sim.rename(columns=name_mapping)
bwfl_ids = [sensor for sensor in sensor_data['bwfl_id'].unique() if sensor not in ['BW1', 'BW4']]
datetime = cl_df['datetime'].unique()

# compute train mse
train_mse = 0
test_mse = 0
for name in bwfl_ids:
    sim = cl_sim[name].values
    data = cl_df.loc[cl_df['bwfl_id'] == name, 'mean'].values
    train_mask = ~np.isnan(sim) & ~np.isnan(data) & (np.arange(len(sim)) >= 96) & (np.arange(len(sim)) < n_train)
    train_mse += (1 / (len(datetime[96:n_train]) * len(bwfl_ids))) * np.sum((sim[train_mask] - data[train_mask]) ** 2)
    test_mask = ~np.isnan(sim) & ~np.isnan(data) & (np.arange(len(sim)) >= n_train)
    test_mse += (1 / (len(datetime[n_train:]) * len(bwfl_ids))) * np.sum((sim[test_mask] - data[test_mask]) ** 2)

print(f"Train MSE: {train_mse}")
# assert train_mse == hof[0].fitness.values[0], "GA train mse is not the same as a posteriori computation."
print(f"Test MSE: {test_mse}")

Save GA results to master spreadsheet.

In [ ]:
save_results = True
if save_results:
    results_df = pd.read_excel(RESULTS_DIR / 'wq/ga_calibration.xlsx', sheet_name=grouping)
    
    results_df.loc[results_df['data_period'] == data_period, 'train_mse'] = train_mse
    results_df.loc[results_df['data_period'] == data_period, 'test_mse'] = test_mse
    results_df.loc[results_df['data_period'] == data_period, 'cpu_time'] = cpu_time
    results_df.loc[results_df['data_period'] == data_period, 'demand_resolution'] = demand_resolution
    results_df.loc[results_df['data_period'] == data_period, 'bulk_coeff'] = wn_train.options.reaction.bulk_coeff * 3600 * 24
    for i, (key, val) in enumerate(wall_coeffs_opt.items()):
        results_df.loc[results_df['data_period'] == data_period, f"wall_coeff_{i}"] = val

    with pd.ExcelWriter(RESULTS_DIR / 'wq/ga_calibration.xlsx', mode='a', if_sheet_exists='replace') as writer:
        results_df.to_excel(writer, sheet_name=grouping, index=False)

### Plot results

Time series plots.

In [ ]:
bwfl_ids = [bwfl_id for bwfl_id in cl_df['bwfl_id'].unique() if bwfl_id not in ['BW1', 'BW4']]
subplot_titles = [f"Chlorine time series @ {bwfl_id}" for bwfl_id in bwfl_ids]
fig = make_subplots(rows=len(bwfl_ids), cols=1, subplot_titles=subplot_titles)

for idx, bwfl_id in enumerate(bwfl_ids): 
    data = cl_df[(cl_df['bwfl_id'] == bwfl_id)]
    sim = cl_sim[bwfl_id].values
    show_legend = (idx == 0)

    # sensor data
    fig.add_trace(
        go.Scatter(
            x=data['datetime'][96:],
            y=data['mean'][96:],
            mode='lines',
            name='actual',  # Legend entry for Sensor series
            line=dict(color='black'),
            showlegend=show_legend
        ),
        row=idx + 1, col=1
    )
    
    # simulated data
    fig.add_trace(
        go.Scatter(
            x=data['datetime'][96:],
            y=sim[96:],
            mode='lines',
            name='model',
            line=dict(color=default_colors[1]),
            showlegend=show_legend
        ),
        row=idx + 1, col=1
    )
    fig.update_yaxes(title_text="Chlorine [mg/L]", rangemode="tozero", row=idx + 1, col=1)

fig.update_layout(
    height=450 * len(bwfl_ids),
    template='simple_white',
    legend_title_text='',
)

fig.show()

Residual boxplots.

In [ ]:
bwfl_ids = [bwfl_id for bwfl_id in cl_df['bwfl_id'].unique() if bwfl_id not in ['BW1', 'BW4']]

residuals_df = pd.DataFrame(columns=['bwfl_id', 'Residuals', 'Dataset'])
for bwfl_id in bwfl_ids:
    data = cl_df[cl_df['bwfl_id'] == bwfl_id]
    sim = cl_sim[bwfl_id].values
    train_residuals = sim[96:n_train] - data['mean'].values[96:n_train]
    test_residuals = sim[n_train:] - data['mean'].values[n_train:]
    
    residuals_df = pd.concat([
        residuals_df,
        pd.DataFrame({'bwfl_id': bwfl_id, 'Residuals': train_residuals, 'Dataset': 'Train'}),
        pd.DataFrame({'bwfl_id': bwfl_id, 'Residuals': test_residuals, 'Dataset': 'Test'})
    ], ignore_index=True)

fig = px.box(residuals_df, x='bwfl_id', y='Residuals', color='Dataset',
             labels={'bwfl_id': 'BWFL ID', 'Residuals': 'Residuals'},
             points='outliers')

fig.update_layout(
    height=450,
    xaxis_title='',
    yaxis_title='Chlorine residual [mg/L]',
    template='simple_white',
    legend_title_text='',
)
fig.show()

Network residuals plot.

In [ ]:
# insert code here...